In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math
from scipy.stats import norm
from scipy.stats import skew, kurtosis
!pip install openpyxl

In [2]:
data = pd.read_csv('Data.csv')
data['date'] = pd.to_datetime(data['date'].astype(str), format='%Y%m%d').dt.to_period('M')


In [3]:

data.info()
#这里注意gvkey是int，所以后面index的时候，注意是int，不是string

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569476 entries, 0 to 569475
Data columns (total 52 columns):
 #   Column     Non-Null Count   Dtype    
---  ------     --------------   -----    
 0   gvkey      569476 non-null  int64    
 1   iid        569476 non-null  int64    
 2   tic        569476 non-null  object   
 3   cusip      569476 non-null  object   
 4   ajexm      569187 non-null  float64  
 5   prccm      564996 non-null  float64  
 6   prchm      564465 non-null  float64  
 7   prclm      564465 non-null  float64  
 8   trfm       565281 non-null  float64  
 9   trt1m      564066 non-null  float64  
 10  cshom      206410 non-null  float64  
 11  fyearq     551792 non-null  float64  
 12  fqtr       551775 non-null  float64  
 13  acoq       419788 non-null  float64  
 14  apq        435411 non-null  float64  
 15  atq        456104 non-null  float64  
 16  ceqq       478674 non-null  float64  
 17  cheq       447212 non-null  float64  
 18  cogsq      462558 non-nu

In [4]:
data.groupby('gvkey').count()
#可以看出来一共1685个股票， 51个features

,iid,tic,cusip,ajexm,prccm,prchm,prclm,trfm,trt1m,cshom,...,NUMUP,NUMDOWN,FY_1,LTG,actual,surpmean,surpstdev,lpermno,sp500,date
gvkey,,,,,,,,,,,,,,,,,,,,,
1010,269,269,269,269,269,269,269,269,268,0,...,0,0,0,0,0,0,0,269,269,269
1013,382,382,382,382,382,382,382,382,381,152,...,66,66,66,66,22,22,21,382,382,382
1040,286,286,286,286,285,285,285,285,284,0,...,19,19,19,19,0,0,0,286,286,286
1043,351,351,351,351,351,351,351,351,350,0,...,0,0,0,0,0,0,0,351,351,351
1045,681,681,681,681,657,657,657,657,656,245,...,410,410,410,395,102,103,100,681,681,681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265975,86,86,86,86,86,86,86,86,85,0,...,0,0,0,0,0,0,0,86,86,86
266101,72,72,72,72,72,72,72,72,71,0,...,0,0,0,0,0,0,0,72,72,72
294524,120,120,120,120,119,119,119,119,119,119,...,119,119,119,115,39,40,37,120,120,120


In [5]:
#所有股票中，最长的日期是705个
max(data.groupby('gvkey').count()['iid'])

705

In [7]:
start_point = '1970-01'

In [8]:
end_point = '2019-12'

# Factor 1: 1M price High - 1M price Low HL1M

In [6]:
prccm = data.pivot(index = 'date', columns = 'gvkey', values = 'prccm')
#这里看到是705行，和1685个stocks，所以是正确的
prccm

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1962-01,73.0000,NaN,NaN,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-02,73.2500,NaN,34.250,80.13,NaN,NaN,NaN,NaN,36.63,74.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-03,68.0000,NaN,33.000,71.50,NaN,NaN,NaN,NaN,36.50,77.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-04,71.8799,NaN,27.750,68.25,NaN,NaN,NaN,NaN,36.25,77.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-05,59.6300,NaN,26.625,56.63,NaN,NaN,NaN,NaN,29.75,66.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05,NaN,NaN,NaN,NaN,10.50,NaN,15.30,NaN,77.90,94.92,...,66.34,15.04,19.28,43.98,NaN,NaN,NaN,63.76,99.70,18.33
2020-06,NaN,NaN,NaN,NaN,13.07,NaN,17.52,NaN,73.29,91.43,...,64.96,15.63,19.01,45.22,NaN,NaN,NaN,65.72,102.22,18.73
2020-07,NaN,NaN,NaN,NaN,11.12,NaN,23.56,NaN,83.08,100.64,...,72.98,14.98,20.23,43.81,NaN,NaN,NaN,62.52,99.46,18.16


In [9]:
#然后选中需要的时间，这里是为了掩饰方法正确，后面就直接一步了
prccm = prccm.loc[start_point:'2019-12']
prccm

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,46.375,NaN,17.625,50.000,23.750,NaN,29.875,17.000,19.375,73.000,...,NaN,NaN,NaN,NaN,NaN,72.000,36.875,NaN,NaN,NaN
1970-02,48.125,NaN,21.000,44.250,29.875,NaN,33.125,16.250,22.125,75.625,...,NaN,NaN,NaN,NaN,NaN,70.000,40.250,NaN,NaN,NaN
1970-03,49.250,NaN,21.500,46.875,26.875,NaN,41.375,16.875,21.750,73.250,...,NaN,NaN,NaN,NaN,NaN,71.375,NaN,NaN,NaN,NaN
1970-04,45.500,NaN,19.875,34.500,23.875,NaN,38.500,10.000,20.125,69.000,...,NaN,NaN,NaN,NaN,NaN,66.500,NaN,NaN,NaN,NaN
1970-05,38.500,NaN,19.250,27.500,24.125,NaN,37.625,8.000,19.000,62.250,...,NaN,NaN,NaN,NaN,NaN,59.750,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,NaN,NaN,26.310,NaN,13.300,NaN,95.310,85.320,...,76.61,26.38,37.99,52.27,270.74,NaN,NaN,77.38,96.27,20.26
2019-09,NaN,NaN,NaN,NaN,26.970,NaN,12.020,NaN,97.070,83.670,...,79.62,33.16,38.68,53.01,259.17,NaN,NaN,89.47,103.65,22.04
2019-10,NaN,NaN,NaN,NaN,30.060,NaN,12.730,NaN,94.120,83.610,...,76.69,31.07,40.40,53.55,296.60,NaN,NaN,89.70,116.04,23.06


In [10]:
prchm = data.pivot(index = 'date', columns = 'gvkey', values = 'prchm').loc[start_point:'2019-12']
prclm = data.pivot(index = 'date', columns = 'gvkey', values = 'prclm').loc[start_point:'2019-12']

In [11]:
HL1M = (prchm - prccm) / (prccm - prclm)

In [12]:
HL1M

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,14.000000,NaN,15.000000,inf,inf,NaN,2.411765,5.500000,11.500000,inf,...,NaN,NaN,NaN,NaN,NaN,1.000000,7.666667,NaN,NaN,NaN
1970-02,0.066667,NaN,0.111111,5.538462,0.028169,NaN,0.000000,1.000000,0.043478,0.333333,...,NaN,NaN,NaN,NaN,NaN,inf,0.037037,NaN,NaN,NaN
1970-03,0.571429,NaN,0.090909,0.354839,2.181818,NaN,0.081967,0.428571,0.818182,0.833333,...,NaN,NaN,NaN,NaN,NaN,1.181818,NaN,NaN,NaN,NaN
1970-04,2.000000,NaN,1.461538,3.714286,3.555556,NaN,1.217391,inf,23.000000,2.350000,...,NaN,NaN,NaN,NaN,NaN,9.500000,NaN,NaN,NaN,NaN
1970-05,2.304348,NaN,0.285714,2.727273,0.317073,NaN,15.250000,1.583333,2.222222,1.333333,...,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,NaN,NaN,2.045673,NaN,0.271084,NaN,0.099332,0.691919,...,1.377301,9.780105,3.158915,1.284404,2.345382,NaN,NaN,0.816990,2.514970,4.122449
2019-09,NaN,NaN,NaN,NaN,3.303922,NaN,17.100000,NaN,0.256367,1.041509,...,0.153430,0.066961,2.181892,1.717033,17.200000,NaN,NaN,0.067992,0.117581,0.376518
2019-10,NaN,NaN,NaN,NaN,0.290090,NaN,0.032862,NaN,1.650485,0.227451,...,2.349476,0.639535,0.103343,0.139875,0.105750,NaN,NaN,0.301685,0.062925,0.094527


In [13]:
HL1M.count()
#注意这里有全是NAH的值

gvkey
1010      173
1013      382
1040      190
1043      255
1045      577
         ... 
265975     16
266101      2
294524    111
316056     73
326688     20
Length: 1685, dtype: int64

In [14]:
#HL1M = HL1M.dropna(axis = 1, how = 'all')
#HL1M

# Factor 2: Expected LTG

In [15]:
LTG = data.pivot(index = 'date', columns = 'gvkey', values = 'LTG').loc[start_point:'2019-12']

In [16]:
#LTG = LTG.dropna(axis = 1, how = 'all')
#LTG

# Factor 3: Long-Term Momentum MOM

In [17]:
help(pd.DataFrame.rolling)

Help on function rolling in module pandas.core.generic:

rolling(self, window: 'int | timedelta | BaseOffset | BaseIndexer', min_periods: 'int | None' = None, center: 'bool_t' = False, win_type: 'str | None' = None, on: 'str | None' = None, axis: 'Axis' = 0, closed: 'str | None' = None, step: 'int | None' = None, method: 'str' = 'single') -> 'Window | Rolling'
    Provide rolling window calculations.
    
    Parameters
    ----------
    window : int, offset, or BaseIndexer subclass
        Size of the moving window.
    
        If an integer, the fixed number of observations used for
        each window.
    
        If an offset, the time period of each window. Each
        window will be a variable sized based on the observations included in
        the time-period. This is only valid for datetimelike indexes.
        To learn more about the offsets & frequency strings, please see `this link
        <https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-al

In [18]:
trt1m = data.pivot(index = 'date', columns = 'gvkey', values = 'trt1m').loc[start_point:'2019-12']

In [59]:
MOM = trt1m.shift(2).rolling(window = 11).mean()
#rolling: the output will be put at the end of the window, so shift 2 position at the beginning
MOM.iloc[0:60]

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
trt1m.loc[start_point:"1970-11",1010].mean()
# Just to prove the calculation is correct and on the correct position 

-0.06182727272727268

In [27]:
trt1m.loc['1970-2':"1970-12",1010].mean()

0.6029363636363638

In [28]:
trt1m.columns

Int64Index([  1010,   1013,   1040,   1043,   1045,   1057,   1062,   1070,
              1075,   1078,
            ...
            189491, 198058, 199356, 260774, 260778, 265975, 266101, 294524,
            316056, 326688],
           dtype='int64', name='gvkey', length=1685)

# Factor 4: Book to Price BP

In [29]:
ceqq = data.pivot(index = 'date', columns = 'gvkey', values = 'ceqq').loc[start_point:'2019-12']
cshoq = data.pivot(index = 'date', columns = 'gvkey', values = 'cshoq').loc[start_point:'2019-12']

In [30]:
BP = ceqq / (cshoq * prccm)

In [31]:
BP

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,0.736925,NaN,0.571365,NaN,0.867679,NaN,1.124589,1.370687,NaN,0.237035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02,0.710128,NaN,0.479539,NaN,0.689786,NaN,1.014252,1.433950,NaN,0.228807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03,0.693907,NaN,0.468387,NaN,0.766786,NaN,0.772810,1.380841,NaN,0.236226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04,NaN,NaN,NaN,NaN,NaN,NaN,0.830519,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05,NaN,NaN,NaN,NaN,NaN,NaN,0.849834,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,NaN,NaN,-0.001878,NaN,0.884131,NaN,0.488908,0.210127,...,0.209749,0.580499,0.297687,0.304132,0.337944,NaN,NaN,0.374068,0.073417,0.740693
2019-09,NaN,NaN,NaN,NaN,-0.001832,NaN,1.337446,NaN,0.480044,0.214271,...,0.201819,0.461808,0.292377,0.299887,0.353031,NaN,NaN,0.323521,0.068190,0.680873
2019-10,NaN,NaN,NaN,NaN,0.012151,NaN,1.262852,NaN,0.525124,0.215182,...,0.208951,0.510482,0.292712,0.306559,0.325415,NaN,NaN,0.263754,0.063462,0.657976


# Factor 5: CAPM Beta

In [32]:
SP500 = pd.read_csv('SP500.csv')

SP500['excess_return'] = SP500['ret_sp500'] - SP500['rf']

SP500['Date'] = pd.to_datetime(SP500['Date'].astype(str),format='%Y%m').dt.to_period('M')

SP500 = SP500.set_index('Date')

rf = SP500['rf'].loc[start_point:'2019-12']

SP500_excess = SP500.drop(['ret_sp500','rf'], axis = 1).loc[start_point:'2019-12']
SP500_excess

,excess_return
Date,
1970-01,-0.081398
1970-02,0.053321
1970-03,-0.002894
1970-04,-0.093831
1970-05,-0.059989
...,...
2019-08,-0.017685
2019-09,0.016991
2019-10,0.020121


In [33]:
rf

Date
1970-01    0.0060
1970-02    0.0062
1970-03    0.0057
1970-04    0.0050
1970-05    0.0053
            ...  
2019-08    0.0016
2019-09    0.0018
2019-10    0.0015
2019-11    0.0012
2019-12    0.0014
Freq: M, Name: rf, Length: 600, dtype: float64

In [34]:
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

In [35]:
stock_excess = trt1m.sub(rf, axis=0) /100

stock_excess

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,-0.053631,NaN,-0.047357,-0.180388,-0.227702,NaN,0.086304,-0.000060,-0.080996,-0.042217,...,NaN,NaN,NaN,NaN,NaN,0.028511,-0.012560,NaN,NaN,NaN
1970-02,0.050612,NaN,0.204193,-0.108062,0.266254,NaN,0.108725,-0.044180,0.141873,0.035897,...,NaN,NaN,NaN,NaN,NaN,-0.013951,0.091463,NaN,NaN,NaN
1970-03,0.023320,NaN,0.023753,0.059265,-0.100475,NaN,0.249000,0.046097,-0.017006,-0.031462,...,NaN,NaN,NaN,NaN,NaN,0.019586,NaN,NaN,NaN,NaN
1970-04,-0.076192,NaN,-0.075631,-0.264050,-0.111678,NaN,-0.069536,-0.407458,-0.062349,-0.054316,...,NaN,NaN,NaN,NaN,NaN,-0.068351,NaN,NaN,NaN,NaN
1970-05,-0.140713,NaN,-0.020179,-0.192807,0.018795,NaN,-0.013689,-0.200053,-0.055954,-0.097879,...,NaN,NaN,NaN,NaN,NaN,-0.086519,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,NaN,NaN,-0.134398,NaN,0.124244,NaN,0.044821,-0.020452,...,-0.042861,-0.258797,-0.139540,-0.013976,-0.057492,NaN,NaN,-0.075413,-0.070230,-0.182751
2019-09,NaN,NaN,NaN,NaN,0.025068,NaN,-0.096259,NaN,0.018448,-0.019357,...,0.039272,0.256995,0.018145,0.014139,-0.042753,NaN,NaN,0.169793,0.079446,0.087840
2019-10,NaN,NaN,NaN,NaN,0.114557,NaN,0.059053,NaN,-0.030405,0.003092,...,-0.033800,-0.063043,0.044452,0.010172,0.144408,NaN,NaN,0.002556,0.119522,0.054205


In [36]:
stock_excess = stock_excess.dropna(axis = 1, how = 'all')

In [37]:
Beta = {}

In [38]:
for col in stock_excess.columns: 
    y = stock_excess[col]
    x = sm.add_constant(SP500_excess)
    rols = RollingOLS(y, x, window=48)
    rres = rols.fit()
    beta = rres.params['excess_return'].copy()
    Beta[str(col)] = beta

/opt/conda/lib/python3.10/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/conda/lib/python3.10/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/conda/lib/python3.10/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/conda/lib/python3.10/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/conda/lib/python3.10/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = ssr / (nobs - tot_params)
/opt/conda/lib/python3.10/site-packages/statsmodels/regression/rolling.py:255: RuntimeWarning: divide by zero encountered in double_sca

In [39]:
Beta = pd.DataFrame(Beta)
Beta.columns = Beta.columns.astype(int) #统一格式，方便后续处理

In [40]:
Beta.head(48)

,1010,1013,1040,1043,1045,1062,1070,1075,1078,1098,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
Date,,,,,,,,,,,,,,,,,,,,,
1970-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Factor 6: Log Market Cap LogMCap

In [44]:
cshom = data.pivot(index = 'date', columns = 'gvkey', values = 'cshom').loc[start_point:'2019-12']

In [45]:
LogMCap = pd.DataFrame(np.log(prccm*cshom))

/opt/conda/lib/python3.10/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [46]:
LogMCap

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,NaN,NaN,23.184143,NaN,19.362861,NaN,23.093855,25.739183,...,23.347356,22.109224,22.293076,23.590044,23.334913,NaN,NaN,23.979103,22.919302,21.954732
2019-09,NaN,NaN,NaN,NaN,23.208919,NaN,19.261669,NaN,23.112152,25.719655,...,23.385894,22.337962,22.311076,23.604102,23.291238,NaN,NaN,24.124279,22.993165,22.038943
2019-10,NaN,NaN,NaN,NaN,23.301057,NaN,19.319059,NaN,23.081291,25.718937,...,23.348400,22.272860,22.354583,23.614237,23.426457,NaN,NaN,24.126846,23.101152,22.084396


# Factor 7: 12M Realized Price Volatility AnnVol12M

In [47]:
prccm

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,46.375,NaN,17.625,50.000,23.750,NaN,29.875,17.000,19.375,73.000,...,NaN,NaN,NaN,NaN,NaN,72.000,36.875,NaN,NaN,NaN
1970-02,48.125,NaN,21.000,44.250,29.875,NaN,33.125,16.250,22.125,75.625,...,NaN,NaN,NaN,NaN,NaN,70.000,40.250,NaN,NaN,NaN
1970-03,49.250,NaN,21.500,46.875,26.875,NaN,41.375,16.875,21.750,73.250,...,NaN,NaN,NaN,NaN,NaN,71.375,NaN,NaN,NaN,NaN
1970-04,45.500,NaN,19.875,34.500,23.875,NaN,38.500,10.000,20.125,69.000,...,NaN,NaN,NaN,NaN,NaN,66.500,NaN,NaN,NaN,NaN
1970-05,38.500,NaN,19.250,27.500,24.125,NaN,37.625,8.000,19.000,62.250,...,NaN,NaN,NaN,NaN,NaN,59.750,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,NaN,NaN,26.310,NaN,13.300,NaN,95.310,85.320,...,76.61,26.38,37.99,52.27,270.74,NaN,NaN,77.38,96.27,20.26
2019-09,NaN,NaN,NaN,NaN,26.970,NaN,12.020,NaN,97.070,83.670,...,79.62,33.16,38.68,53.01,259.17,NaN,NaN,89.47,103.65,22.04
2019-10,NaN,NaN,NaN,NaN,30.060,NaN,12.730,NaN,94.120,83.610,...,76.69,31.07,40.40,53.55,296.60,NaN,NaN,89.70,116.04,23.06


In [48]:
def annvol(window):
    return math.sqrt(sum(window) / 11) * math.sqrt(12)

In [49]:
AnnVol12M = prccm.rolling(window = 13).apply(annvol)

In [51]:
AnnVol12M.iloc[0:60]

gvkey,1010,1013,1040,1043,1045,1057,1062,1070,1075,1078,...,189491,198058,199356,260774,260778,265975,266101,294524,316056,326688
date,,,,,,,,,,,,,,,,,,,,,
1970-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Sorting and Caculating Q spread

In [52]:
factors_data = {'HL1M':HL1M , 'LTG':LTG, 'MOM':MOM, 'BP':BP, 'Beta':Beta, 'LogMCap':LogMCap, 'AnnVol12M':AnnVol12M}
final_results = {}

In [54]:
trt1m_new = data.pivot(index = 'date', columns = 'gvkey', values = 'trt1m').loc[start_point:'2020-01'] 
#因为最后t+1 会到2020-01，所以要创建一个新的对应的return表

In [55]:
factor_keys = ['HL1M','LTG','MOM', 'BP', 'Beta', 'LogMCap', 'AnnVol12M']

In [56]:
def find_factor_return(factor_transposed):
    #example: find_factor_return(HL1M_transposed)
    
    
    factor_return = pd.Series() #创建一个存放结果的series
    
    for i in range(len(factor_transposed.columns)):
        t = factor_transposed.columns[i] 
        each_period = factor_transposed.loc[:,t].sort_values(ascending = False)
        #拿出transpose之后的每一个column， 也就是在t时间所有公司的这个factor的数据，然后sort
        
        selected = each_period[~ each_period.isin([np.nan, np.inf, -np.inf, 0])] #去掉正负无穷，去掉nan，去掉0
    
        top_20_percent = selected[:int(len(selected)*0.2)]
        bottom_20_percent = selected[int(len(selected)*0.8):]
    
        R_winner = trt1m_new.loc[t+1].loc[top_20_percent.index].mean()
        R_loser = trt1m_new.loc[t+1].loc[bottom_20_percent.index].mean()
    
        QSpread = R_winner - R_loser
    
        factor_return.loc[t] = QSpread
    
    return factor_return

In [57]:
for i in factor_keys:
    factor = factors_data[i].dropna(axis = 1, how = 'all') #这里要drop一次，因为要不然transpose之后 each_period会是empty
    factor_transposed = factor.T.dropna(axis = 1, how = 'all') #这里也是同理，刚才是竖着，现在是transpose之后再竖着一次
    
    factor_return = find_factor_return(factor_transposed)
    final_results[i] = factor_return

/tmp/ipykernel_145/815761247.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  factor_return = pd.Series() #创建一个存放结果的series
/tmp/ipykernel_145/815761247.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  factor_return = pd.Series() #创建一个存放结果的series
/tmp/ipykernel_145/815761247.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  factor_return = pd.Series() #创建一个存放结果的series
/tmp/ipykernel_145/815761247.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  factor_return = pd.Series() #创建一个存放结果的series
/tmp/ipykernel_145/815761247

In [58]:
final_results = pd.DataFrame(final_results)
final_results

,HL1M,LTG,MOM,BP,Beta,LogMCap,AnnVol12M
1970-01,3.418607,NaN,NaN,4.096120,NaN,NaN,NaN
1970-02,1.572903,NaN,NaN,2.587761,NaN,NaN,NaN
1970-03,-1.805106,NaN,NaN,-0.053737,NaN,NaN,NaN
1970-04,-0.283914,NaN,NaN,-0.879979,NaN,NaN,NaN
1970-05,3.952885,NaN,NaN,-4.576180,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2019-08,6.082182,-6.221058,-10.086998,5.426739,4.340075,-7.848948,-7.253780
2019-09,0.140758,0.080944,-1.937100,-2.813529,2.946011,1.102345,1.130526
2019-10,-2.499045,-0.323316,-1.177283,-0.752505,6.484734,2.026085,2.245392
2019-11,2.973642,-1.458998,-6.094281,6.827667,4.254100,-4.880016,-5.888711


In [ ]:
'''bug = HL1M_transposed.loc[:,'2002-01'].sort_values().dropna()

bug.loc[:bug.index[-1]]

bug[np.isinf(bug)].empty'''

# Single factor situation

In [286]:
HL1M_transposed = HL1M.T

HL1M_transposed

#HL1M_transposed.apply(lambda col: col.sort_values().reset_index(drop=True), axis=0)

HL1M_return = pd.Series()
#创建对应的factor return result series

for i in range(len(HL1M_transposed.columns)):
    t = HL1M_transposed.columns[i]
    each_period = HL1M_transposed.loc[:,t].sort_values(ascending = False) #.dropna()
    
    
    selected = each_period[~each_period.isin([np.nan, np.inf, -np.inf, 0])]
    
    
    #bullshit，没有考虑全
    '''inf_series = each_period[np.isinf(each_period)]
    nan_series = each_period[each_period != 0]
    
    start = each_period.index[0] if inf_series.empty else inf_series.index[-1]
    end = each_period.index[-1] if nan_series.empty else nan_series.index[-1]
    #这里是怕如果整个series中没有inf的值，那么这个inf series会是empty，所以index就检索不到，会报错
    
    if inf_series.empty: 
        start = each_period.index[0]
    else:
        start = inf_series.index[-1]
    
    if nan_series.empty:
        end = each_period.index[-1]
    else:
        end = nan_series.index[-1]
    
    start = each_period[np.isinf(each_period)].index[-1]
    end = each_period[each_period != 0].index[-1]'''
    
    #selected = each_period.loc[start:end].iloc[1:]
    
    top_20_percent = selected[:int(len(selected)*0.2)]
    bottom_20_percent = selected[int(len(selected)*0.8):]
    
    R_winner = trt1m_new.loc[t+1].loc[top_20_percent.index].mean()
    R_loser = trt1m_new.loc[t+1].loc[bottom_20_percent.index].mean()
    
    QSpread = R_winner - R_loser
    
    HL1M_return.loc[t] = QSpread

HL1M_return

/tmp/ipykernel_45/3731325700.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  HL1M_return = pd.Series()


1987-01    1.240300
1987-02    1.459551
1987-03   -0.701074
1987-04    0.296679
1987-05    1.850123
             ...   
2019-08    6.082182
2019-09    0.140758
2019-10   -2.499045
2019-11    2.973642
2019-12    6.992718
Freq: M, Length: 396, dtype: float64

In [237]:
Beta#.dropna(axis = 1, how = 'all')

,1013,1043,1045,1062,1075,1078,1098,1111,1161,1164,...,187128,188255,189491,198058,199356,260774,260778,294524,316056,326688
Date,,,,,,,,,,,,,,,,,,,,,
1987-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1987-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08,NaN,NaN,1.902596,-0.086141,0.075982,1.114190,NaN,NaN,3.261891,NaN,...,2.392511,1.732671,1.196403,0.945453,1.241468,1.713138,0.905107,1.480566,1.195367,1.724735
2019-09,NaN,NaN,1.942450,-0.125713,0.116984,1.054146,NaN,NaN,3.233127,NaN,...,2.365471,1.766694,1.217056,0.977726,1.212108,1.738997,0.869656,1.512191,1.200320,1.743118
2019-10,NaN,NaN,1.891888,-0.121835,0.137428,1.014356,NaN,NaN,3.358284,NaN,...,2.570141,1.829271,1.205928,1.188686,0.874150,1.700018,0.979490,1.512761,1.180028,1.751691


# draft below

In [280]:
t = HL1M_transposed.columns[0]
t

Period('1987-01', 'M')

In [281]:
first_col = HL1M_transposed.loc[:,t].sort_values(ascending = False)
#first_col.index = first_col.index.astype(str)
first_col#.loc[9380:7724]

gvkey
9061      inf
9777      inf
4560      inf
11135     inf
12950     inf
         ... 
265975    NaN
266101    NaN
294524    NaN
316056    NaN
326688    NaN
Name: 1987-01, Length: 1685, dtype: float64

In [282]:
selected = first_col[~first_col.isin([np.nan, np.inf, -np.inf, 0])]

In [270]:
end = first_col[first_col != 0].index[-1]
end

7724

In [271]:
#end = min(first_col[first_col.isna()].index[0], first_col[np.isinf(first_col)].index[0])
start = first_col[np.isinf(first_col)].index[-1]

In [272]:
start

4016

In [273]:
selected = first_col.loc[start:end]#.iloc[1:]
selected

gvkey
4016           inf
5458     53.000000
1567     40.000000
4700     30.000000
11649    25.000000
           ...    
9514      0.015038
10540     0.014925
6304      0.013514
8215      0.011905
7724      0.010417
Name: 1987-01, Length: 803, dtype: float64

In [283]:
top_20_percent = selected[:int(len(selected)*0.2)]
top_20_percent

gvkey
5458     53.000000
1567     40.000000
4700     30.000000
11649    25.000000
7238     22.500000
           ...    
4710      1.031250
4050      1.016129
4732      1.000000
10097     1.000000
6799      1.000000
Name: 1987-01, Length: 160, dtype: float64

In [284]:
bottom_20_percent = selected[int(len(selected)*0.8):]
bottom_20_percent

gvkey
2146     0.155172
4108     0.153846
7650     0.153846
1837     0.153846
9699     0.153846
           ...   
9514     0.015038
10540    0.014925
6304     0.013514
8215     0.011905
7724     0.010417
Name: 1987-01, Length: 161, dtype: float64

In [285]:
top_20_percent.index

Int64Index([ 5458,  1567,  4700, 11649,  7238, 10412,  8873,  5667,  5060,
            10519,
            ...
             2663,  2489,  8470,  3054,  1318,  4710,  4050,  4732, 10097,
             6799],
           dtype='int64', name='gvkey', length=160)

In [121]:
bottom_20_percent.index

Int64Index([ 2146,  4108,  7650,  1837,  9699,  6268,  9611, 12603,  7154,
             2424,
            ...
             5439, 11502,  6223,  7127,  8247,  9514, 10540,  6304,  8215,
             7724],
           dtype='int64', name='gvkey', length=161)

In [139]:
R_winner = trt1m.loc[t+1].loc[top_20_percent.index].mean()
R_winner

6.975161875

In [140]:
R_loser = trt1m.loc[t+1].loc[bottom_20_percent.index].mean()
R_loser

5.73486149068323

In [133]:
QSpread = R_winner - R_loser
QSpread 

1.2403003843167708

In [144]:
HL1M_return.loc[t] = QSpread

In [145]:
HL1M_return

1987-01    1.2403
Freq: M, dtype: float64

In [ ]:
HL1M['1987-01'][HL1M['1987-01'] != 0].idxmin()

In [210]:
s = pd.Series([0, 0, 0, 4, 5, 0, 6])
first_non_zero = s[s != 0].idxmax()

In [211]:
first_non_zero

6

In [229]:
s = pd.Series([1, 2, np.inf, -np.inf, 5])

print(s)

# Check for infinite values
print(np.isinf(s))

0    1.0
1    2.0
2    inf
3   -inf
4    5.0
dtype: float64
0    False
1    False
2     True
3     True
4    False
dtype: bool


In [201]:
first_quintile = df.quantile(0.2)  # Top 20%
bottom_quintile = df.quantile(0.8)  # Bottom 20%

In [202]:
first_quintile

A    2.0
B    5.0
C    8.0
Name: 0.2, dtype: float64